In [1]:
import cv2
import numpy as np

#cap = cv2.VideoCapture("img/people.mp4")

cap = cv2.VideoCapture(0)

#########################################################################
#########################################################################
### webcam den nesne algılama yapmak istiyorsak #########################
### cap = cv2.VideoCapture(0) yazmamız yeterli ##########################
#########################################################################
#########################################################################


while True:
    ret, frame = cap.read()
    
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    
    frame_blob = cv2.dnn.blobFromImage(frame, 1/255, (416,416), swapRB = True, crop = False)
    
    labels = ["person", "bicycle","car", "motorcycle", "airplane", "bus","train","truck","boat", 
              "trafficlight","firehydrant", "stopsign", "parkingmeter", "bench", "bird","cat", 
              "dog", "horse","sheep", "cow","elephant", "bear", "zebra", "giraffe", "backpack",
              "umbrella", "handbag", "tie","suitcase","frisbee", "skis", "snowboard","sportsball", 
              "kite", "baseballbat", "baseballglove", "skateboard", "surfboard", "tennisracket", 
              "bottle", "wineglass", "cup","fork", "knife","spoon", "bowl","banana", "apple", 
              "sandwich", "orange", "broccoli", "carrot", "hotdog","pizza","donut", "cake", "chair",
              "sofa", "pottedplant", "bed", "diningtable","toilet", "tvmonitor", "Laptop","mouse", 
              "remote", "keyboard", "cellphone", "microwave","oven", "toaster", "sink","refrigerator",
              "book","clock", "vase","scissors", "teddybear", "hairdrier","toothbrush"]
    
    
    colors = ["0,255,255", "0,0,255", "255,255,0", "0,255,0"]
    colors = [np.array(color.split(",")).astype("int") for color in colors]
    colors = np.array(colors)
    colors = np.tile(colors,(18,1))  
    
    
    model = cv2.dnn.readNetFromDarknet("model/yolov3.cfg", "model/yolov3.weights")
    
    layers = model.getLayerNames()

   
    output_layers = [layers[layer[0]-1] for layer in model.getUnconnectedOutLayers()]
    
    model.setInput(frame_blob)

    detection_layers = model.forward(output_layers)

    #################### NON_MAXIMUM SUPPRESSION _ OPERATION 1 ####################################
    ids_list = []
    boxes_list = []
    confidence_list = []

    ################### END OF OPERATION 1 #######################################################

    for detection_layer in detection_layers:
        for object_detection in detection_layer:
            scores = object_detection[5:]
            predicted_id = np.argmax(scores)
            confidence = scores[predicted_id]

            if confidence > 0.20:
                label = labels[predicted_id]
                bounding_box = object_detection[0:4] * np.array([frame_width, frame_height, frame_width, frame_height ])
                (box_center_x, box_center_y, box_width, box_height) = bounding_box.astype("int")

                start_x = int(box_center_x - (box_width/2))
                start_y = int(box_center_y - (box_height/2))

                ################NON-MAXIMUM SUPPREASSION - OPERATION 2 ############################



                ids_list.append(predicted_id)
                confidence_list.append(float(confidence))
                boxes_list.append([start_x, start_y, int(box_width), int(box_height)])





                ############################END OF OPERATION 2 #######################################

             
    ################NON-MAXIMUM SUPPREASSION - OPERATION 3 ############################

    max_ids = cv2.dnn.NMSBoxes(boxes_list, confidence_list, 0.5, 0.4)  #0.5 ve 0.4 değerleri algoritmamıza göre değiştirebiliriz

    for max_id in max_ids:
        max_class_id = max_id[0]
        box = boxes_list[max_class_id]

        start_x = box[0]
        start_y = box[1]
        box_width = box[2]
        box_height = box[3]

        predicted_id = ids_list[max_class_id]
        label = labels[predicted_id]
        confidence = confidence_list[max_class_id]

    ############################END OF OPERATION 3 #######################################


        end_x = start_x + box_width
        end_y = start_y + box_height

        box_color = colors[predicted_id]
        box_color = [int(each) for each in box_color]

        label = "{}: {:.2f}%".format(label, confidence*100)
        print("predicted object {}".format(label)) #güven skorunu ekrana yazdırmak için

        cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), box_color, 2)
        cv2.putText(frame, label, (start_x, start_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)


    cv2.imshow("Detection Window" , frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()           

    

predicted object person: 86.22%
predicted object person: 92.83%
predicted object person: 96.45%
predicted object person: 90.67%
predicted object person: 92.69%
predicted object person: 94.70%
predicted object bottle: 92.16%
predicted object person: 89.36%
predicted object bottle: 99.23%
predicted object person: 98.33%
predicted object bottle: 99.34%
predicted object person: 92.39%
predicted object bottle: 95.84%
predicted object bottle: 98.12%
predicted object person: 85.94%
predicted object bottle: 93.38%
predicted object person: 87.28%
predicted object parkingmeter: 67.24%
predicted object bottle: 92.50%
predicted object person: 59.78%
predicted object person: 69.78%
predicted object bottle: 58.16%
predicted object person: 53.29%
predicted object mouse: 77.48%
predicted object mouse: 81.55%
predicted object Laptop: 99.86%
predicted object Laptop: 99.44%
predicted object Laptop: 99.64%
predicted object Laptop: 99.68%
predicted object Laptop: 99.87%
predicted object person: 84.77%
pred